In [1]:
import sys
sys.path.append("../")

import sqlite3
import numpy as np

from src.cmesrc.config import CMESRC_DB
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
con = sqlite3.connect(CMESRC_DB)
con.execute("PRAGMA foreign_keys = ON")
cur = con.cursor()

In [4]:
cur.execute("SELECT timestamp FROM images")

timestamps = cur.fetchall()
#timestamps = timestamps[::30]

In [5]:
def distance_between_bboxes(bboxA, bboxB):
    # Original bbox = [harpnum, xmin, xmax, ymin, ymax, xcen, ycen]
    # Convert to bbox = [xmin, ymin, xmax, ymax]
    bboxA = [bboxA[1], bboxA[3], bboxA[2], bboxA[4]]
    bboxB = [bboxB[1], bboxB[3], bboxB[2], bboxB[4]]

    inner_bbox = [
        max(bboxA[0], bboxB[0]),
        max(bboxA[1], bboxB[1]),
        min(bboxA[2], bboxB[2]),
        min(bboxA[3], bboxB[3])
    ]

    width = inner_bbox[2] - inner_bbox[0]
    height = inner_bbox[3] - inner_bbox[1]

    dist_width = min(0, width)
    dist_height = min(0, height)

    dist = dist_width**2 + dist_height**2

    return dist

In [6]:
cur.execute("DROP TABLE IF EXISTS distance_matrix")
cur.executescript("""
CREATE TABLE IF NOT EXISTS distance_matrix (
    harpnum1 INTEGER NOT NULL,
    harpnum2 INTEGER NOT NULL CHECK (harpnum1 < harpnum2),
    timestamp TEXT NOT NULL,
    dist REAL NOT NULL,
    PRIMARY KEY (harpnum1, harpnum2, timestamp)
    );
""")

In [7]:
cur.execute("CREATE INDEX timestamp_index ON harps_pixel_bbox (timestamp);")

OperationalError: index timestamp_index already exists

In [8]:
con.commit()

In [9]:
i = 0
for timestamp in tqdm(timestamps):
    timestamp = timestamp[0]
    cur.execute("SELECT harpnum, x_min, x_max, y_min, y_max, x_cen, y_cen FROM harps_pixel_bbox WHERE timestamp = ?", (timestamp,))
    bboxes = cur.fetchall()

    from copy import deepcopy
    CUTOUT_SIZE = float(224)
    bboxes_copy = deepcopy(bboxes)
    i = 0
    while len(bboxes_copy) != 0:
        bbox1 = bboxes_copy.pop()

        for bbox2 in bboxes_copy:
            harpnum1 = min(bbox1[0], bbox2[0]) # Smallest harpnum, used for cutout
            harpnum2 = max(bbox1[0], bbox2[0]) # Larger harpnum

            bboxA = [bbox1 if bbox1[0] == harpnum1 else bbox2][0] # Smallest harpnum
            bboxB = [bbox2 if bbox2[0] == harpnum2 else bbox1][0] # Larger harpnum

            dist = distance_between_bboxes(bboxA, bboxB)

            cur.execute("INSERT OR REPLACE INTO distance_matrix (harpnum1, harpnum2, timestamp, dist) VALUES (?, ?, ?, ?)", (harpnum1, harpnum2, timestamp, dist))



    i += 1

    if i % 1000 == 0:
        con.commit()
        

100%|██████████| 429129/429129 [00:49<00:00, 8734.53it/s]  


In [10]:
con.commit()

In [11]:
con.close()

### TESTS

In [9]:
sample_timestamp = "2014-07-30 12:48:00"
sample_timestamp

'2014-07-30 12:48:00'

In [55]:
cur.execute("SELECT harpnum, x_min, x_max, y_min, y_max, x_cen, y_cen FROM harps_pixel_bbox WHERE timestamp = ?", (sample_timestamp,))
bboxes = cur.fetchall()

In [56]:
bboxes

[(4397, 196, 258, 266, 305, 227, 285),
 (4398, 95, 147, 160, 186, 121, 173),
 (4399, 107, 208, 250, 303, 157, 276),
 (4413, 113, 121, 278, 284, 117, 281),
 (4414, 147, 161, 156, 165, 154, 160),
 (4379, 312, 383, 237, 286, 347, 261),
 (4383, 331, 398, 179, 232, 364, 205),
 (4390, 149, 255, 147, 206, 202, 176),
 (4392, 270, 291, 171, 184, 280, 177),
 (4396, 82, 157, 161, 236, 119, 198)]

In [49]:
dmatcon.close()

In [87]:
dmatcon = sqlite3.connect("./notebook_10_files/dmat.db")
dmatcon.execute("PRAGMA foreign_keys = ON")
dmatcur = dmatcon.cursor()

In [169]:
dmatcur.execute("DROP TABLE IF EXISTS distance_matrix")

In [170]:
dmatcur.executescript("""
CREATE TABLE IF NOT EXISTS distance_matrix (
    harpnum1 INTEGER NOT NULL,
    harpnum2 INTEGER NOT NULL,
    timestamp TEXT NOT NULL,
    dist REAL NOT NULL,
    PRIMARY KEY (harpnum1, harpnum2, timestamp)
    );
""")

In [171]:
dmatcur.execute("PRAGMA table_info(distance_matrix)")
print(dmatcur.fetchall())

[(0, 'harpnum1', 'INTEGER', 1, None, 1), (1, 'harpnum2', 'INTEGER', 1, None, 2), (2, 'timestamp', 'TEXT', 1, None, 3), (3, 'dist', 'REAL', 1, None, 0)]


In [172]:
def iou(bboxA, bboxB):
    # Original bbox = [harpnum, xmin, xmax, ymin, ymax, xcen, ycen]
    # Convert to bbox = [xmin, ymin, xmax, ymax]

    harpnum1 = bboxA[0]
    harpnum2 = bboxB[0]

    bboxA = [bboxA[1], bboxA[3], bboxA[2], bboxA[4]]
    bboxB = [bboxB[1], bboxB[3], bboxB[2], bboxB[4]]

    # Intersection coordinates
    xA = max(bboxA[0], bboxB[0])
    yA = max(bboxA[1], bboxB[1])
    xB = min(bboxA[2], bboxB[2])
    yB = min(bboxA[3], bboxB[3])

    inter_area = max(0, xB - xA) * max(0, yB - yA)

    bboxA_area = (bboxA[2] - bboxA[0]) * (bboxA[3] - bboxA[1])
    bboxB_area = (bboxB[2] - bboxB[0]) * (bboxB[3] - bboxB[1])

    union_area = bboxA_area + bboxB_area - inter_area

    iou = float(inter_area) / float(union_area)

    if harpnum1 == 4399 and harpnum2 == 4413:
        print(bboxA, bboxB)
        print(xA, yA, xB, yB)
        print(inter_area, union_area, iou)

    return iou

def cutout_overlap_ratio(bboxA, cutout):
    # Original bbox = [harpnum, xmin, xmax, ymin, ymax, xcen, ycen]
    # Convert to bbox = [xmin, ymin, xmax, ymax]

    harpnum1 = bboxA[0]
    harpnum2 = cutout[0]

    bboxA = [bboxA[1], bboxA[3], bboxA[2], bboxA[4]]
    cutout = [cutout[1], cutout[3], cutout[2], cutout[4]]

    # Intersection coordinates
    xA = max(bboxA[0], cutout[0])
    yA = max(bboxA[1], cutout[1])
    xB = min(bboxA[2], cutout[2])
    yB = min(bboxA[3], cutout[3])

    inter_area = max(0, xB - xA) * max(0, yB - yA)

    bboxA_area = (bboxA[2] - bboxA[0]) * (bboxA[3] - bboxA[1])

    overlap_ratio = float(inter_area) / float(bboxA_area)

    if harpnum1 == 4413 and harpnum2 == 4399:
        print(bboxA, cutout)
        print(xA, yA, xB, yB)
        print(inter_area, bboxA_area)
        print(overlap_ratio)

    return overlap_ratio

def distance_between_bboxes(bboxA, bboxB):
    # Original bbox = [harpnum, xmin, xmax, ymin, ymax, xcen, ycen]
    # Convert to bbox = [xmin, ymin, xmax, ymax]
    bboxA = [bboxA[1], bboxA[3], bboxA[2], bboxA[4]]
    bboxB = [bboxB[1], bboxB[3], bboxB[2], bboxB[4]]

    inner_bbox = [
        max(bboxA[0], bboxB[0]),
        max(bboxA[1], bboxB[1]),
        min(bboxA[2], bboxB[2]),
        min(bboxA[3], bboxB[3])
    ]

    width = inner_bbox[2] - inner_bbox[0]
    height = inner_bbox[3] - inner_bbox[1]

    dist_width = min(0, width)
    dist_height = min(0, height)

    dist = dist_width**2 + dist_height**2

    return dist

In [173]:
%%time
distance_between_bboxes(bboxes[7], bboxes[2])

CPU times: user 3 µs, sys: 2 µs, total: 5 µs
Wall time: 7.87 µs


1936

In [180]:
from copy import deepcopy
CUTOUT_SIZE = float(224)
bboxes_copy = deepcopy(bboxes)
i = 0
while len(bboxes_copy) != 0:
    bbox1 = bboxes_copy.pop()

    for bbox2 in bboxes_copy:
        harpnum1 = min(bbox1[0], bbox2[0]) # Smallest harpnum, used for cutout
        harpnum2 = max(bbox1[0], bbox2[0]) # Larger harpnum

        bboxA = [bbox1 if bbox1[0] == harpnum1 else bbox2][0] # Smallest harpnum
        bboxB = [bbox2 if bbox2[0] == harpnum2 else bbox1][0] # Larger harpnum

        dist = distance_between_bboxes(bboxA, bboxB)

        dmatcur.execute("INSERT OR REPLACE INTO distance_matrix (harpnum1, harpnum2, timestamp, dist) VALUES (?, ?, ?, ?)", (harpnum1, harpnum2, sample_timestamp, dist))

        i += 1
print(i)

dmatcon.commit()

45


In [187]:
dmatcur.execute("SELECT * FROM distance_matrix WHERE (harpnum1 = 4392 OR harpnum2 = 4392) AND dist = 0.0 LIMIT 10")
#dmatcur.execute("SELECT * FROM distance_matrix LIMIT 10")
res = dmatcur.fetchall()

for r in res:
    print(r)

In [179]:
len(bboxes)

10

In [15]:
# Print column names of images table

cur.execute("PRAGMA table_info(images)")
print(cur.fetchall())

[(0, 'timestamp', 'TEXT', 1, None, 0), (1, 'year', 'INTEGER', 1, None, 0), (2, 'month', 'INTEGER', 1, None, 0), (3, 'day', 'INTEGER', 1, None, 0), (4, 'hour', 'INTEGER', 1, None, 0), (5, 'minute', 'INTEGER', 1, None, 0), (6, 'second', 'INTEGER', 1, None, 0), (7, 'idx', 'INTEGER', 1, None, 0)]
